# AI4Code Pytorch DistilBert Baseline

I used a lot of code from Kaggle's starter notebook here: https://www.kaggle.com/code/ryanholbrook/getting-started-with-ai4code

I replaced their model with a DistilBert model.

In [2]:
%cd ..

/home/briancy2/ai4code-baseline


In [3]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import tqdm
import copy
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers.models.longformer.modeling_longformer import LongformerSelfAttention
from transformers import LongformerModel, LongformerTokenizer, LongformerConfig


In [3]:
import json
from pathlib import Path
from pathlib import Path
from datasets import load_dataset
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch import nn
#from model import *
from tqdm import tqdm
import sys, os
from metrics import *
import torch
import argparse
from stlr import SlantedTriangular
data_dir = str(Path.cwd()) + '/data/processed_dataset_1/'

val_df_mark = pd.read_csv(data_dir+'val_mark.csv').drop("parent_id", axis=1).dropna().reset_index(drop=True)
df_orders = pd.read_csv(
    data_dir + 'train_orders.csv',
   index_col='id',
   squeeze=True,
).str.split() 
def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    inputs = torch.nn.utils.rnn.pad_sequence([ t[0] for t in batch], batch_first=True)
    masks = torch.nn.utils.rnn.pad_sequence([ t[1] for t in batch], batch_first=True)
    fts = torch.nn.utils.rnn.pad_sequence([ t[2] for t in batch], batch_first=True)
    ranks = torch.nn.utils.rnn.pad_sequence([ t[3] for t in batch], batch_first=True)
    
    return inputs, masks, fts, ranks

test_fts = json.load(open(data_dir+'val_fts.json'))
test_df = pd.read_csv(data_dir+'val.csv').drop("parent_id", axis=1).dropna().reset_index(drop=True)

/tmp/ipykernel_3286703/453387531.py:19: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  df_orders = pd.read_csv(


In [4]:
def get_features(df):
    df = df.sort_values("rank").reset_index(drop=True)
    for idx, sub_df in tqdm(df.groupby("id")):
        n_md = sub_df[sub_df.cell_type == "markdown"].shape[0]
        n_code = sub_df[sub_df.cell_type == "code"].shape[0]
        df.loc[sub_df.index, "fts"] =  0 if n_md + n_code == 0 else n_md / (n_md + n_code)
        df.loc[sub_df.index, "pct_rank"] = df["rank"] / (n_md + n_code)
    return df
test_df = get_features(test_df)

100%|██████████| 13964/13964 [00:44<00:00, 311.48it/s]


In [5]:
test_df

,id,cell_id,cell_type,source,rank,ancestor_id,pct_rank,fts
0,398c274073bc3c,f267d8f9,markdown,machine learning micro course home page http w...,0,e74027ef,0.000000,0.400000
1,ad199a5a21da98,b4e580ff,markdown,branch 1 create 3 different vanilla network tr...,0,bc97c28f,0.000000,0.342105
2,ad133e6f301825,d7ef540d,markdown,header2 import library,0,117dae91,0.000000,0.250000
3,ad108a03ad1b3d,de953e0f,markdown,header1 hello fellow novice hey decided notebo...,0,dcc24880,0.000000,0.391753
4,40251d4da356af,7f2a64ec,markdown,header2 curent notebook focus covid 19 related...,0,f603cddf,0.000000,0.256410
...,...,...,...,...,...,...,...,...
629190,785f21b8226d02,392b7c75,code,y_test test iloc 31 y_test y_test reset_index ...,750,b6063675,0.993377,0.058278
629191,785f21b8226d02,51da1043,code,common y_test merge y_pred left_index true rig...,751,b6063675,0.994702,0.058278
629192,785f21b8226d02,e2291da9,code,mat confusion_matrix y_test y_pred sns heatmap...,752,b6063675,0.996026,0.058278
629193,785f21b8226d02,045395f4,code,print classification_report y_test y_pred,753,b6063675,0.997351,0.058278


In [6]:
max_len = 512
md_max_len = 64
n_codes_sampled = 40
code_sum_len = max_len - md_max_len
code_max_len = 23
re_ranking_thereshold = 40

In [7]:
from tqdm import tqdm
import sys, os
from transformers import AutoModel, AutoTokenizer, RobertaForMaskedLM
import torch.nn.functional as F
import torch.nn as nn
import torch

class MarkdownModel(nn.Module):
    def __init__(self, model_path):
        super(MarkdownModel, self).__init__()
        self.attention_window = 512
        self.md_max_len = 64
        self.model = RobertaForMaskedLM.from_pretrained(model_path).roberta
        self.dense = nn.Linear(769, 769)
        self.dropout = nn.Dropout(0.1)
        self.top = nn.Linear(769, 1)

    def forward(self, ids, mask, fts):
        x = self.model(input_ids=ids, attention_mask=mask)[0]
        x = torch.cat((x[:, 0, :], fts), 1)
        x = self.top(x)
        return x
           

In [8]:
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModel, AutoTokenizer, RobertaForMaskedLM
def sample_codes(lst: list) -> list:

    n_codes = len(lst)
    if n_codes <= n_codes_sampled:
        return lst
    else:
        return [lst[no] for no in [round(n * n_codes / n_codes_sampled) for n in range(n_codes_sampled)]]

def filte_codes_for_validation(n_codes: int, pre_ranking_pred: int) -> list:

    if n_codes <= n_codes_sampled:
        return range(n_codes)
    elif pre_ranking_pred <= n_codes_sampled / 2:
        return range(n_codes_sampled)
    elif pre_ranking_pred >= n_codes - n_codes_sampled / 2 + 1:
        return range(n_codes - n_codes_sampled, n_codes)
    else:
        return range(math.floor(pre_ranking_pred - n_codes_sampled / 2), math.floor(pre_ranking_pred + n_codes_sampled / 2))

class ValidSet(Dataset):

    """
    code_source_df: row no. of notebook -> list of code source
    n_codes_lst: row no. of markdown -> number of code cells in the corresponding notebook
    nb_no_lst: row no. of markdown -> row no. of notebook
    md_no_lst: idx of dataset -> row no. of markdown
    code_no_lst: row no. of markdown -> list of no. of used codes
    """

    def __init__(self, df, model_name):

        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

        self.md_df = df[df['cell_type'] == 'markdown'].reset_index(drop = True)
        code_df = df[df['cell_type'] == 'code']

        self.code_source_df = (
            code_df
            .groupby('id')['source']
            .apply(list)
            .to_frame()
        )

        self.n_codes_lst = (
            self.md_df['id']
            .map(code_df['id'].value_counts())
        )

        self.nb_no_lst = (
            self.md_df['id']
            .map({id: no for no, id in enumerate(self.code_source_df.index)})
        )
        self.fts = df["fts"]
        self.pct_ranks = df["pct_rank"]
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def __getitem__(self, idx):

        md_embed = self.tokenizer.encode_plus(
            self.md_df['source'][idx],
            max_length = md_max_len,
            padding = 'max_length',
            truncation = True
        )

        ids, mask = md_embed['input_ids'], md_embed['attention_mask']

        code_source = [self.code_source_df['source'][self.nb_no_lst[idx]][no] for no in self.code_no_lst[idx]]
        code_embed = self.tokenizer.batch_encode_plus(
            code_source,
            max_length = code_max_len,
            padding = 'max_length',
            truncation = True
        )

        for code_ids in code_embed['input_ids']:
            ids.extend(code_ids[:-1])
        for code_mask in code_embed['attention_mask']:
            mask.extend(code_mask[:-1])

        ids.extend([self.tokenizer.pad_token_id] * (max_len - len(ids)))
        mask.extend([0] * (max_len - len(mask)))
        
        fts = self.fts[idx]
        pct_rank = self.pct_ranks[idx]

        return torch.LongTensor(ids), torch.LongTensor(mask), torch.FloatTensor([fts]), torch.FloatTensor([pct_rank])

    def __len__(self):

        return self.md_df.shape[0]


class ValidSetForPreRanking(ValidSet):

    def __init__(self, df, model_name):
        super().__init__(df, model_name)
        
        self.code_no_lst = [sample_codes(range(n_codes)) for n_codes in self.n_codes_lst]


class ValidSetForReRanking(ValidSet):

    def __init__(self, df, model_name, pre_ranking_preds):
        super().__init__(df, model_name)

        self.code_no_lst = [filte_codes_for_validation(n_codes, pred) for n_codes, pred in zip(self.n_codes_lst, pre_ranking_preds)]

In [22]:
from tqdm import tqdm
from collections import OrderedDict

def read_data(data):
    return tuple(d.cuda() for d in data[:-1]), data[-1].cuda()

def pred(model, val_loader):
    model.eval()
    
    tbar = tqdm(val_loader, file=sys.stdout)
    
    preds = []
    labels = []

    with torch.no_grad():
        for idx, data in tqdm(enumerate(tbar)):
            inputs, target = read_data(data)
            fts = inputs[2]
            
            model.cuda()
            pred = model(*inputs)

            preds.append(pred.detach().cpu().numpy().ravel())
            labels.append(target.detach().cpu().numpy().ravel())  
    return np.concatenate(labels), np.concatenate(preds)       

def valid(preds):

    valid_set.md_df['cell_order'] = preds
    code_df = test_df[test_df['cell_type'] == 'code'].reset_index()
    code_df['cell_order'] = code_df.groupby('id')['index'].rank()
    
    return (
        valid_set
        .md_df
        .append(code_df)
        .sort_values('cell_order')
        .groupby('id')['cell_id']
        .apply(' '.join)
    )


model_path = "microsoft/codebert-base-mlm"
ckpt_path = "./outputs/preprocess/model-4.bin"

valid_set = ValidSetForPreRanking(test_df, model_path)

valid_loader = DataLoader(
    valid_set,
    batch_size = 64,
    num_workers = 2
)

model = MarkdownModel(model_path).cuda() # for clasification
state_dict = torch.load(ckpt_path)
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
    # load params
model.load_state_dict(new_state_dict, strict=True)  
_, pre_ranking_preds = pred(model,valid_loader)
test_df.loc[test_df["cell_type"] == "markdown", "pred"] = pre_ranking_preds * 0.96
# df_2 = test_df.loc[test_df["cell_type"] == "markdown"].sort_values("pred").groupby('id').apply(list)

#val_df.loc[val_df["cell_type"] == "markdown", "pred"] = y_pred 
#y_dummy = val_df.sort_values("pred").groupby('id')['cell_id']
#print("Preds score", kendall_tau(df_orders.loc[y_dummy.index], df_2))


# new_order = test_df.loc[test_df["cell_type"] == "markdown"][["id","cell_id","pred"]]
# #print(new_order)
# for g in tqdm(df_2):
#     g[1].reset_index(inplace=True)
#     cells = new_order.loc[new_order['id'] == g[0]]
#     for cell_id in g[1]["cell_id"]:
#         cells[cells["cell_id"] == cell_id]["pred"]  = g[1][g[1]["cell_id"]==cell_id].index.to_numpy()
# preds_for_rerank = new_order["pred"].to_numpy()
# valid_set = ValidSetForReRanking(test_df,model_path, preds_for_rerank) # for re-ranking
# valid_loader = DataLoader(
#     valid_set,
#     batch_size = 64,
#     num_workers = 2
# )
# _, preds = pred(model, valid_loader)
# print(preds)
# df_chenm = valid(preds)
# df_chenm.name = 'cell_order'
# df_chenm.to_csv('submission.csv')

  0%|          | 0/3367 [00:00<?, ?it/s]


0it [00:00, ?it/s]
1it [00:01,  1.17s/it]

  0%|          | 1/3367 [00:01<1:05:49,  1.17s/it]


2it [00:01,  1.20it/s]

  0%|          | 2/3367 [00:01<47:16,  1.19it/s]  

...


3366it [34:39,  1.62it/s]

100%|█████████▉| 3366/3367 [34:39<00:00,  1.62it/s]


3367it [34:39,  1.72it/s]

100%|██████████| 3367/3367 [34:39<00:00,  1.62it/s]

3367it [34:39,  1.62it/s]

In [27]:
pre_ranking_preds

array([0.05284828, 0.07931383, 0.07804927, ..., 0.14302635, 0.14411347,
       0.14523076], dtype=float32)

In [23]:
y_dummy = test_df.loc[test_df["cell_type"] == "markdown"].sort_values("pred").groupby('id')['cell_id'].apply(list)

In [26]:
test_df.loc[test_df["cell_type"] == "markdown"]

,id,cell_id,cell_type,source,rank,ancestor_id,pct_rank,fts,pred
0,398c274073bc3c,f267d8f9,markdown,machine learning micro course home page http w...,0,e74027ef,0.000000,0.400000,0.050734
1,ad199a5a21da98,b4e580ff,markdown,branch 1 create 3 different vanilla network tr...,0,bc97c28f,0.000000,0.342105,0.076141
2,ad133e6f301825,d7ef540d,markdown,header2 import library,0,117dae91,0.000000,0.250000,0.074927
3,ad108a03ad1b3d,de953e0f,markdown,header1 hello fellow novice hey decided notebo...,0,dcc24880,0.000000,0.391753,0.048995
4,40251d4da356af,7f2a64ec,markdown,header2 curent notebook focus covid 19 related...,0,f603cddf,0.000000,0.256410,0.054087
...,...,...,...,...,...,...,...,...,...
629063,785f21b8226d02,e6f6bf47,markdown,header2 neural network 15 hidden layer 500 epo...,623,b6063675,0.825166,0.058278,0.140131
629090,785f21b8226d02,6da12b6b,markdown,header2 neural network 3 hidden layer 500 epoc...,650,b6063675,0.860927,0.058278,0.140108
629117,785f21b8226d02,eac79c74,markdown,header2 neural network 5 hidden layer 500 epoc...,677,b6063675,0.896689,0.058278,0.137305
629143,785f21b8226d02,00206965,markdown,header2 neural network 10 hidden layer 500 epo...,703,b6063675,0.931126,0.058278,0.138349


In [24]:
print("Preds score", kendall_tau(df_orders.loc[y_dummy.index], y_dummy))

Preds score 0.882600724024044


In [11]:
df_2

id
000b8e6d58544b    [id, cell_id, cell_type, source, rank, ancesto...
000c0a9b2fef4d    [id, cell_id, cell_type, source, rank, ancesto...
0010adf10cf27c    [id, cell_id, cell_type, source, rank, ancesto...
00116ebc2a301c    [id, cell_id, cell_type, source, rank, ancesto...
00181d9eb98d2c    [id, cell_id, cell_type, source, rank, ancesto...
                                        ...                        
ffe023cbafd709    [id, cell_id, cell_type, source, rank, ancesto...
ffe3e4945bd64f    [id, cell_id, cell_type, source, rank, ancesto...
fff0000b99af52    [id, cell_id, cell_type, source, rank, ancesto...
fff994f45b92c0    [id, cell_id, cell_type, source, rank, ancesto...
fffb5392db4c39    [id, cell_id, cell_type, source, rank, ancesto...
Length: 13961, dtype: object

In [146]:
# model_path = "../input/codebert-base-mlm/codebert-base-mlm/"
# ckpt_path = "../input/ai4code-models/model-4.bin"
# y_test_2 = predict(model_path, ckpt_path)
# test_df.loc[test_df["cell_type"] == "markdown", "pred"] = y_test_2
# df_2 = test_df.sort_values("pred").groupby("id")["cell_id"].apply(lambda x: " ".join(x)).reset_index()
# df_2.rename(columns={"cell_id": "cell_order"}, inplace=True)

In [147]:
# df_2.to_csv('submission.csv', index = False)